<style>
@font-face {font-family: "B Nazanin"; src: url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.eot"); src: url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.eot?#iefix") format("embedded-opentype"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.woff2") format("woff2"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.woff") format("woff"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.ttf") format("truetype"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.svg#B Nazanin") format("svg"); }
    </style>
<div dir="rtl" align="center" style ='font-family: "B Nazanin";'>
    <h1>
        تمرین چهارم درس پردازش زبان‌های طبیعی
    </h1>
    <h3>
        گردآورندگان:<br/>
        ساحل مس‌فروش، سروش تابش، درنا دهقانی
    </h3>
    
</div>

<br/>
<br/>


<div dir="rtl"  style ='font-family: "B Nazanin";'>
    ما در این تمرین ترک "فن بیان تیتر" را انتخاب کردیم. در این تمرین قصد داریم به کمک دو روش، متون را بر اساس جذاب بودن یا نبودن تیتر آنها طبقه‌بندی کنیم. در روش اول از یک ترنسفورمر از پیش آموزش دیده و در روش دوم از CNN استفاده می‌کنیم.
</div>

In [19]:
import json
import tqdm
import torch
import re
import random

<div dir="rtl"  style ='font-family: "B Nazanin";'>
    ابتدا دیتای داده شده را  بر اساس یادگیری، ارزیابی و تست تقسیم می‌کنیم. با توجه به اینکه این ترک بر اساس تیتر عمل می‌کند و متن خبر تاثیری ندارد، تنها این بخش از دیتا و بیان تیتر آن را بازنویسی می‌کنیم.
</div>

In [2]:
def read_data(path, state=None):
    data = json.load(open(path))
    if state is not None:
        data = data.get(state)
    return data

In [3]:
path = "./dataset_annotated_title_fluent.json"
# train_data = read_data(path, "train")

In [4]:
def extract_title(text):
    regex = re.compile("\\n.*\\n")
    title = re.split(regex, text)[0]
    return title


In [5]:
def rewrite_data(path):
    data = read_data(path)
    for key, value in data.items():
        with open(f"{key}.json", "w", encoding="utf-8") as save_file:
            new_result = list()
            for data_item in value:
                data_item["text"] = extract_title(data_item.get("text"))
                new_result.append(data_item)
            final_res = {key: new_result}
            # print(final_res)
            json.dump(final_res, save_file, ensure_ascii=False)
            del new_result
                

In [7]:
rewrite_data(path)

<div dir="rtl"  style ='font-family: "B Nazanin";'>
    برای مشاهده‌ی ساختار داده‌ها، داده‌ی یادگیری را از فایل json می‌خوانیم و به dataframe تبدیل می‌کنیم. هم‌چنین اخباری که تیتر یا فن بیان نامشخص دارند را حذف می‌کنیم.
</div>

In [154]:
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [152]:
import pandas as pd

path = './train.json'
data = read_data(path)
data = data['train']
df = pd.DataFrame(data)
df = df[df['annotations'].map(lambda d: len(d)) > 0]
df = df[df['text'] != ''].reset_index(drop=True)
df

,text,annotations
0,آتش سوزی کارخانه جمیل نخ گسترده است / اعزام نی...,"[بیان عادی, بیان جذاب, بیان عادی]"
1,تجهیزات باکیفیت صنعتی آشپزخانه و کافی شاپ,"[بیان عادی, بیان عادی, بیان عادی]"
2,برگزاری جشنواره رسانه ابوذر در دی ماه سال جاری,"[بیان عادی, بیان عادی, بیان عادی]"
3,افزایش 80 درصدی فروش سلاح در آمریکا در ژانویه ...,"[بیان عادی, بیان عادی, بیان عادی]"
4,بازداشت دو تروریست که در بمب گذاری مسیر زائران...,"[بیان عادی, بیان عادی]"
...,...,...
1439,پیش بینی پیر جماران!(یادداشت روز),"[بیان جذاب, بیان جذاب, بیان جذاب]"
1440,افتتاح یک باب مدرسه خیرساز در فاریاب,"[بیان عادی, بیان عادی, بیان عادی]"
1441,چرا وزارت نفت درباره مطالبه راه اندازی دفتر نف...,[بیان جذاب]
1442,اطلاع رسانی عجیب پرسپولیس از تمرین امروز,"[بیان عادی, بیان عادی, بیان عادی]"


<div dir="rtl"  style ='font-family: "B Nazanin";'>
  حال به افزودن ستون‌های مورد نیاز به dataframe می‌پردازیم. با توجه به وجود چند رای برای فن بیان هر تیتر، برای نمایش آن احتمال عادی و جذاب بودن تیتر را نسبت به کل نظرها حساب می‌کنیم.
</div>

In [153]:
NORMAL = 'بیان عادی'
INTERESTING = 'بیان جذاب'

def label_prob(labels):
  normal_counter = 0
  for l in labels:
    if l == NORMAL:
      normal_counter += 1
  return normal_counter / len(labels)
 

In [35]:
# in case of need to vote

def label_vote(labels):
  if all(x == labels[0] for x in labels):
    return labels[0]
  normal_counter = 0
  interesting_counter = 0
  for l in labels:
    if l == NORMAL:
      normal_counter += 1
      if normal_counter > 1:
        return NORMAL
    elif l == INTERESTING:
      interesting_counter += 1
      if interesting_counter > 1:
        return INTERESTING
  return random.choice([NORMAL, INTERESTING])

<div dir="rtl"  style ='font-family: "B Nazanin";'>
    سپس به تمیز کردن تیتر اخبار می‌پردازیم. در این بخش ابتدا آن را نرمال‌سازی کرده، سپس علائم و اعداد اضافه آن را پاک می‌کنیم. نهایتاً الگوهایی مانند emojiها و symbolها نیز از آن حذف می‌گردد.
</div>

In [94]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Found existing installation: cleantext 1.1.4
Uninstalling cleantext-1.1.4:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/cleantext-1.1.4.dist-info/*
    /usr/local/lib/python3.7/dist-packages/cleantext/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/cleantext/clean.py
    /usr/local/lib/python3.7/dist-packages/cleantext/constants.py
    /usr/local/lib/python3.7/dist-packages/cleantext/sklearn.py
    /usr/local/lib/python3.7/dist-packages/cleantext/specials.py
    /usr/local/lib/python3.7/dist-packages/cleantext/utils.py
Proceed (y/n)? y
  Successfully uninstalled cleantext-1.1.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [143]:
import hazm

def cleaning(text):
  text = text.strip()
  normalizer = hazm.Normalizer()
  text = normalizer.normalize(text)

  # removing additional characters
  text = re.sub("\d", "", text)
  text = text.lower()
  text = re.sub("[a-z]", "", text)
  text = re.sub('/|\+|\||\-|؛|،', ' ', text)

  # may be needed: ((  (,),",",«,»,:,!  ))
  
    
  # removing wierd patterns
  wierd_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        # u"\u200c"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)
    
  text = wierd_pattern.sub(r'', text)
    
  # removing extra spaces, hashtags
  text = re.sub("\s+", " ", text)
  return text

<div dir="rtl"  style ='font-family: "B Nazanin";'>
    سپس به tokenize کردن تیتر می‌پردازیم. برای این کار از tokenizer parsBert استفاده کردیم.
</div>

In [147]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 36.0 MB/s 
     |████████████████████████████████| 101 kB 7.9 MB/s 
     |████████████████████████████████| 596 kB 61.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [148]:
from transformers import AutoTokenizer

model_name_or_path = "HooshvareLab/bert-fa-zwnj-base"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
def token_title(text):
  return tokenizer.tokenize(text)

Downloading:   0%|          | 0.00/292 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

<div dir="rtl"  style ='font-family: "B Nazanin";'>
    از توابع نوشته شده برای اضافه کردن ستون‌های موردنیاز و باز نویسی آن استفاده می‌کنیم.
</div>

In [160]:
files = ['train', 'eval', 'test']
for f in files:
  # read from json
  p = f'./{f}.json'
  data = read_data(p)
  data = data[f]
  df = pd.DataFrame(data)
  # remove empty news
  df = df[df['annotations'].map(lambda d: len(d)) > 0]
  df = df[df['text'] != ''].reset_index(drop=True)
  # get probs of title type
  df['normal_prob'] = df['annotations'].apply(label_prob)
  df['interesting_prob'] = 1 - df['normal_prob']
  # cleaning
  df['clean'] = df['text'].apply(cleaning)
  # tokenizing
  df['tokens'] = df['clean'].apply(token_title)
  # save csv
  df.to_csv(f'{f}_clean.csv', encoding='utf-8-sig')
  print(f'{f} is done')

train is done
eval is done
test is done
